In [1]:
import pandas as pd
import geopandas as gpd

In [82]:
#main raw file which includes all Items
fao_raw=pd.read_excel('fao/FAO production 1992-2018 (11 pollination crops for NCA project - global data - TWN_adj).xlsx')

In [83]:
fao_raw.head()

,Area Code,Area,Country,Item Code,Item,Year,Value
0,2,Afghanistan,AFG,515,Apples,1992,17200.0
1,2,Afghanistan,AFG,515,Apples,1993,17000.0
2,2,Afghanistan,AFG,515,Apples,1994,17500.0
3,2,Afghanistan,AFG,515,Apples,1995,19669.0
4,2,Afghanistan,AFG,515,Apples,1996,23341.0


In [4]:
fao_raw.dtypes

Area Code      int64
Area          object
Country       object
Item Code      int64
Item          object
Year           int64
Value        float64
dtype: object

In [5]:
fao_raw['Item Code'].unique()

array([221, 711, 515, 526, 569, 568, 534, 521, 536, 328, 289, 267, 567,
       176, 414, 181, 531, 530, 220, 401, 397, 399, 497, 430, 394, 236,
       544, 495, 388, 689, 507, 242, 270, 661, 249, 217, 656, 773, 254,
       571, 572, 339, 600, 280, 423, 195, 550, 592, 292, 587, 547,  89,
       554, 197, 692, 263, 224, 687, 216, 203])

In [74]:
#shortName=pd.read_csv('fao/country_shortname_V1.csv',usecols=[0, 1,2])
shortName=pd.read_excel('fao/country_short_name_with_percantege.xlsx',usecols=[0,1,2,6,7,8,9])

In [75]:
shortName.dtypes

Area                             object
Country                          object
Area Code                        object
Split_Year                      float64
What_happened                    object
Country_split_(by_area_code)     object
Percentages                      object
dtype: object

In [76]:
shortName.head()

,Area,Country,Area Code,Split_Year,What_happened,Country_split_(by_area_code),Percentages
0,Armenia,ARM,1,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,2,NaN,NaN,NaN,NaN
2,Albania,ALB,3,NaN,NaN,NaN,NaN
3,Algeria,DZA,4,NaN,NaN,NaN,NaN
4,American Samoa,ASM,5,NaN,NaN,NaN,NaN


In [77]:
len(fao_raw['Item Code'].unique())

60

#### FAO Excel export according to crop type

With below code first, we split fao excel by crop and pivot this table according to year
and put the value of each product under year columns which names are "1992", "1993" .etc and then
merge with shortName file to calculate seperated country statistic. For this calculation, we use weight info that in shortName file.

Last prdouct will be merged with world shp and use for pollination model for NCA project.

In [63]:
output_folder='polinatior_model_from_fao'
for i in fao_raw['Item Code'].unique():
   
    # with this for loop, we export each product from fao excel file
    # i means indiviual item
    
    # we get the item name
    item_name=fao_raw[fao_raw['Item Code']==i].Item.values[0]
    '''
    # we used this block when we want to manipulate name differently
    if len(item_name.split(','))>1 or len(item_name.split(' '))>1 :
        # with this blog, if item_name include more than 1 word,
        # we get the first word and update item_name variable
        if len(item_name.split(','))>1:
            #item_name=(item_name.split(','))[0]
            item_name=item_name.replace(',','_')
            item_name=item_name.replace(' ','_')
        else:
            #item_name=(item_name.split(' '))[0]
            item_name=item_name.replace(' ','_')
            item_name=item_name.replace(',','_')
    '''
    item_name=item_name.replace(',','_')
    item_name=item_name.replace(' ','_')
    # get the target item into new dataframe 
    product=fao_raw[fao_raw['Item Code']==i]
    # with pivot method, we convert the all year row into column value
    pivot_product=pd.pivot_table(product,index=['Area Code',"Area",'Item Code','Item'],columns='Year',values=['Value']).reset_index()
    #create new column name
    columns=[]
    for j,k in pivot_product.columns:
        if j!='Value':
            columns.append(j)
        if k!='':
            columns.append(k)
    #update column name
    pivot_product.columns=columns
    #pivot_product['Area Code']=pivot_product['Area Code'].astype('str')
    # we get country short name info from shortName csv file and merge with pivot_product
    last_product=pd.merge(shortName,pivot_product,left_on='Area Code',right_on='Area Code',how='inner')
    # drop duplicate column and rename the Area column
    last_product.drop(columns=['Area_y'],inplace=True)
    last_product.rename(columns={'Area_x':'Area'},inplace=True)
    #export data to target folder with iten_name
    last_product.columns=last_product.columns.astype(str)
    for index,row in last_product.iterrows():
        if not pd.isna(row['Country_split_(by_area_code)']):
            split_by_area_code=row['Country_split_(by_area_code)'].split(',')
            split_year=int(row['Year'])
            percentage=row['Percentages'].split(',')
            #print(split_by_area_code)
            for index2,area_code in enumerate(split_by_area_code):
                #try:
                pd.options.mode.chained_assignment = None 
                target_row=last_product[last_product['Area Code']==area_code]
                #print(target_row.loc[:,'1992':str(split_year-1)])
                up=row['1992':str(split_year-1)].apply(lambda x:x*float(percentage[index2]))
                #print(up)
                for i in up.index:
                    target_row.loc[:,i]=up.loc[i]
                    #print(target_row.loc[:,'1992':str(split_year-1)])

                last_product.update(target_row)
                #print('-------------')
                #print(target_row.loc[:,'1992':str(split_year-1)])
            last_product.drop(index, inplace=True)
    last_product.to_csv(f'{output_folder}/pollinator_{item_name}_fao.csv',index=False)

#### FAO excel export according to year

We use same idea as above code. The difference between two code is last product seperated by year. Item names(Apple, Maize) are going to be column name. Also we changed the last calcualtion for seperated country.

In [84]:
output_folder='polinatior_model_from_fao_v2'
#for i in fao_raw['Item Code'].unique():
for y in fao_raw['Year'].unique():
    # with this for loop, we export each product from fao excel file
    # y means indiviual year
    
    #create dataframe according to year
    byYear_target_item=fao_raw[fao_raw['Year']==y]
    
    #create empty dataframe
    yearly_df=pd.DataFrame(columns=['Area Code','Area','Country'])
    for i in byYear_target_item['Area Code'].unique():
        # get each country stastic into dataframe
        byCountry_target=byYear_target_item[byYear_target_item['Area Code']==i]
        # create dataframe with country fundemental inforamtion: Area Code, Country, Area
        target_df=byCountry_target.iloc[:1,0:3]
        for item in byCountry_target['Item Code'].unique():
            item_name=byCountry_target[byCountry_target['Item Code']==item].Item.values[0]
            item_name=item_name.replace(',','_')
            item_name=item_name.replace(' ','_')
            item_name=item_name.replace('__','_')
            item_df=byCountry_target[byCountry_target['Item Code']==item]
            
            #add year information into row
            target_df['Year']=int(y)
            target_df[item_name]=item_df['Value'].values[0]
            
        yearly_df=yearly_df.append(target_df)
    
    yearly_df.reset_index(inplace=True,drop=True)
    yearly_df['Area Code']=yearly_df['Area Code'].astype('str')
    shortName['Area Code']=shortName['Area Code'].astype('str')
    
        
    # we get country short name info from shortName csv file and merge with pivot_product
    last_product=pd.merge(shortName,yearly_df,left_on='Area Code',right_on='Area Code',how='inner')
    # drop duplicate column and rename the Area column
    last_product.drop(columns=['Area_y'],inplace=True)
    last_product.drop(columns=['Country_y'],inplace=True)
    last_product.rename(columns={'Area_x':'Area'},inplace=True)
    last_product.rename(columns={'Country_x':'Country'},inplace=True)
    last_product.columns=last_product.columns.astype(str)
    
    index_list=[]
    for index,row in last_product.iterrows():
        
        #if row include seperation information code starts
        if not pd.isna(row['Country_split_(by_area_code)']):            
            split_by_area_code=row['Country_split_(by_area_code)'].split(',')
            split_year=int(row['Split_Year'])
            if int(y)<int(split_year):                          
                percentage=row['Percentages'].split(',')
                for index2,area_code in enumerate(split_by_area_code):
                    pd.options.mode.chained_assignment = None 
                    target_row=shortName[shortName['Area Code']==str(area_code)]
                    df_row=row.to_frame()
                    for index3,value in enumerate(row[8:],8):
                        new_value=float(row[index3])*float(percentage[index2])
                        target_row.loc[:,'Year']=int(y)
                        target_row.loc[:,df_row.index[index3]]=new_value
                    
                    # add the new row(country) into last product
                    last_product=last_product.append(target_row)
                index_list.append(index)    
    #reset last product index
    last_product=last_product.reset_index(drop=True)    
    for drop_index in index_list:
        last_product.drop(drop_index, inplace=True)
    
    last_product.to_csv(f'{output_folder}/pollinator_{str(y)}_fao.csv',index=False)
    

In [85]:
for u in fao_raw['Item'].unique():
    print(u)

Apples
Melons, other (inc.cantaloupes)
Peaches and nectarines
Pears
Plums and sloes
Watermelons
Cucumbers and gherkins
Pumpkins, squash and gourds
Cocoa, beans
Mangoes, mangosteens, guavas
Avocados


In [86]:
len(fao_raw['Item'].unique())

11